<a href="https://colab.research.google.com/github/ananaymital/Cifar-10_CNN/blob/master/Cifar10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np


Unpickling training and testing data

In [0]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def reshape_rgb_values(x, h=32, w=32):
    return np.reshape([[x[i], x[i+1024], x[i+2048]] for i in range(x.shape[0]//3)], (h, w, 3)).tolist()

def create_image(x):
    w, h = 32, 32
    np_img = reshape_rgb_values(x, h, w)
    img = Image.fromarray(np_img, 'RGB')
    display(img)
    return np_img

Image classes

In [3]:
meta_data_filename = "/content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/batches.meta"

meta_data = unpickle(meta_data_filename)
meta_data

{b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_cases_per_batch': 10000,
 b'num_vis': 3072}

Reading Train files

In [4]:
def read_files(filename=None):
    final_df = pd.DataFrame()
    if filename == None:
        for i in range(1, 6):
            in_filename = "/content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/data_batch_{}".format(i)

            data = unpickle(in_filename)
            df = pd.DataFrame()
            df["data"] = pd.Series(x for x in data.get(b"data"))
            df["labels"] = data.get(b"labels")

            labels = [i.decode("utf-8")  for i in meta_data.get(b"label_names")]
            df["label_names"]=  df.labels.map(dict(enumerate(labels)))

            df.data = df.data.apply(reshape_rgb_values)

            final_df = pd.concat([final_df, df])
            print("loaded {}".format(in_filename))
        return final_df
    else:
        data = unpickle(filename)
        df = pd.DataFrame()
        df["data"] = pd.Series(x for x in data.get(b"data"))
        df["labels"] = data.get(b"labels")

        labels = [i.decode("utf-8")  for i in meta_data.get(b"label_names")]
        df["label_names"]=  df.labels.map(dict(enumerate(labels)))

        df.data = df.data.apply(reshape_rgb_values)
        return df

train = read_files()
train.head()

loaded /content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/data_batch_1
loaded /content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/data_batch_2
loaded /content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/data_batch_3
loaded /content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/data_batch_4
loaded /content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/data_batch_5


,data,labels,label_names
0,"[[[59, 62, 63], [43, 46, 45], [50, 48, 43], [6...",6,frog
1,"[[[154, 177, 187], [126, 137, 136], [105, 104,...",9,truck
2,"[[[255, 255, 255], [253, 253, 253], [253, 253,...",9,truck
3,"[[[28, 25, 10], [37, 34, 19], [38, 35, 20], [4...",4,deer
4,"[[[170, 180, 198], [168, 178, 196], [177, 185,...",1,automobile


Reading Test file

In [0]:
test = read_files("/content/drive/My Drive/ComputerVision/Image classification/data/cifar-10-batches-py/test_batch")

Normalizing pixels and splitting into X_train, y_train, X_test, y_test  

In [6]:
y_train = train.labels.astype("float32")
y_test = test.labels.astype("float32")

X_train = np.asarray(train.data.apply(lambda x: np.asarray(x)/255.0).values.tolist())
X_test = np.asarray(test.data.apply(lambda x: np.asarray(x)/255.0).values.tolist())

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

In [0]:
from tensorflow.nn import fractional_max_pool

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy

Building CNN model

In [0]:
def frac_max_pool(x):
    return fractional_max_pool(x,2**0.5)[0]
    
def build_cnn():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, padding="same"))
    model.add(Lambda(frac_max_pool))
    model.add(Activation("relu"))
    model.add(BatchNormalization())

    model.add(Conv2D(32, kernel_size=3, padding="same"))
    model.add(Lambda(frac_max_pool))
    model.add(Activation("relu"))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Lambda(frac_max_pool))
    model.add(Activation("relu"))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Lambda(frac_max_pool))
    model.add(Activation("relu"))
    model.add(BatchNormalization())

    model.add(Flatten())
    
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    
    model.add(Dense(10, activation="relu"))

    model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model


Deleting train model to reduce memory consumption

In [0]:
try:
    del train
    # del test
except:
    pass

Fitting on X_train & y_train on 25 epochs and validating on X_test & y_test

In [11]:
model = build_cnn()
history = model.fit(x=X_train, y=y_train, epochs=25, validation_data=(X_test, y_test))

Epoch 1/25
1563/1563 [==============================] - 207s 132ms/step - loss: 1.5564 - accuracy: 0.4634 - val_loss: 1.4027 - val_accuracy: 0.5145
Epoch 2/25
1563/1563 [==============================] - 204s 131ms/step - loss: 1.1244 - accuracy: 0.6057 - val_loss: 1.2103 - val_accuracy: 0.5731
Epoch 3/25
1563/1563 [==============================] - 204s 131ms/step - loss: 0.9651 - accuracy: 0.6615 - val_loss: 1.2567 - val_accuracy: 0.5684
Epoch 4/25
1563/1563 [==============================] - 210s 134ms/step - loss: 0.8802 - accuracy: 0.6917 - val_loss: 1.0354 - val_accuracy: 0.6473
Epoch 5/25
1563/1563 [==============================] - 207s 132ms/step - loss: 0.8110 - accuracy: 0.7154 - val_loss: 0.8863 - val_accuracy: 0.6929
Epoch 6/25
1563/1563 [==============================] - 206s 132ms/step - loss: 0.7766 - accuracy: 0.7299 - val_loss: 0.9676 - val_accuracy: 0.6724
Epoch 7/25
1563/1563 [==============================] - 208s 133ms/step - loss: 0.7329 - accuracy: 0.7424 - val_

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  896       
_________________________________________________________________
lambda (Lambda)              multiple                  0         
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
batch_normalization (BatchNo multiple                  128       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  9248      
_________________________________________________________________
lambda_1 (Lambda)            multiple                  0         
_________________________________________________________________
activation_1 (Activation)    multiple                  0

Saving the model

In [0]:
model.save("/content/drive/My Drive/ComputerVision/Image classification/cifar10_cnn.h5")

Deleting model to be loaded later

In [0]:
del model

Loading saved model

In [0]:
from tensorflow import keras

saved_model = keras.models.load_model('/content/drive/My Drive/ComputerVision/Image classification/cifar10_cnn.h5')

Predicting on the test set to analyze confusion matrix and f1 scores

In [28]:
pred = saved_model.predict(X_test)
pred[:5] 

array([[ 0.6225636 ,  0.32338855,  0.        ,  6.871658  ,  0.        ,
         5.499221  ,  4.3128066 ,  0.        ,  0.        ,  0.        ],
       [ 1.9715036 ,  9.124576  ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , 11.988169  ,  0.14353243],
       [ 3.122646  ,  4.396226  ,  1.0907114 ,  1.9308321 ,  0.        ,
         0.        ,  0.        ,  0.        ,  6.3881927 ,  0.33194733],
       [ 6.4478035 ,  3.1872966 ,  1.0387517 ,  0.46368986,  0.        ,
         0.        ,  0.        ,  0.        ,  8.195003  ,  2.6962302 ],
       [ 0.        ,  0.5862502 ,  3.624225  ,  1.1721973 ,  5.371509  ,
         0.        ,  7.9527307 ,  0.        ,  1.7151989 ,  0.        ]],
      dtype=float32)

In [29]:
df_pred = pd.DataFrame(pred)
df_pred = df_pred.apply(lambda x: x.idxmax(), axis=1)
df_pred.head()

0    3
1    8
2    8
3    8
4    6
dtype: int64

In [25]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report

confusion_matrix(y_true=test.labels, y_pred=df_pred)

array([[835,  16,  26,  18,   8,   2,  11,   3,  58,  23],
       [  5, 906,   2,   2,   0,   3,   5,   0,  18,  59],
       [ 68,   9, 703,  36,  36,  33,  91,  10,   8,   6],
       [ 30,  14,  70, 603,  43,  78, 109,  24,  17,  12],
       [ 20,   3, 104,  34, 692,  13, 101,  22,  10,   1],
       [ 13,   6,  66, 151,  46, 618,  68,  18,   9,   5],
       [  6,   2,  43,  24,   7,   3, 905,   1,   5,   4],
       [ 19,   4,  34,  39,  67,  28,  27, 766,   6,  10],
       [ 47,  18,   5,   8,   5,   0,  13,   2, 881,  21],
       [ 21,  78,   4,   6,   1,   0,   3,   6,  29, 852]])

In [26]:
accuracy_score(y_true=test.labels, y_pred=df_pred)

0.7761

In [27]:
f1_score(y_true=test.labels, y_pred=df_pred, average="weighted")

0.7744755203243452

In [30]:
print(classification_report(test.labels, df_pred))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1000
           1       0.87      0.90      0.88      1000
           2       0.68      0.70      0.69      1000
           3       0.66      0.60      0.63      1000
           4       0.76      0.69      0.72      1000
           5       0.78      0.61      0.68      1000
           6       0.66      0.90      0.76      1000
           7       0.88      0.77      0.82      1000
           8       0.88      0.89      0.88      1000
           9       0.84      0.87      0.86      1000

    accuracy                           0.78     10000
   macro avg       0.78      0.78      0.78     10000
weighted avg       0.78      0.78      0.78     10000

